# Parallel programming for CPU programming 

## concurrent.futures

[concurrent.futures](https://docs.python.org/3/library/concurrent.futures.html)

The concurrent.futures modules provides interfaces for running tasks using pools of thread or process workers. The APIs are the same, so applications can switch between threads and processes with minimal changes.

The module provides two types of classes for interacting with the pools. 

```Executors``` are used for managing pools of workers, and ```futures``` are used for managing results computed by the workers. 

To use a pool of workers, an application creates an instance of the appropriate executor class and then submits tasks for it to run. When each task is started, a Future instance is returned. 

When the result of the task is needed, an application can 

### ThreadPoolExecutor

ThreadPoolExecutor manages a set of worker threads, passing tasks to them as they become available for more work. 

This example uses map() to concurrently produce a set of results from an input iterable. The task uses ```time.sleep()``` to pause a different amount of time to demonstrate that, regardless of the order of execution of concurrent tasks, ```map()``` always returns the values in order based on the inputs.

In [7]:
from concurrent import futures
import threading
import time


def task(n):
    print('{}: sleeping {}'.format(
        threading.current_thread().name,
        n)
    )
    time.sleep(n)
    print('{}: done with {}'.format(
        threading.current_thread().name,
        n)
    )
    return n * 10.


#Initiate the threads
ex = futures.ThreadPoolExecutor(max_workers=2)
print('main: starting')

#Start the threads
results = ex.map(task, range(5, 0, -1))

print('main: waiting for real results')
real_results = list(results)
print('main: results: {}'.format(real_results))

main: starting
ThreadPoolExecutor-5_0: sleeping 5ThreadPoolExecutor-5_1: sleeping 4

main: waiting for real results
ThreadPoolExecutor-5_1: done with 4
ThreadPoolExecutor-5_1: sleeping 3
ThreadPoolExecutor-5_0: done with 5
ThreadPoolExecutor-5_0: sleeping 2
ThreadPoolExecutor-5_0: done with 2
ThreadPoolExecutor-5_0: sleeping 1
ThreadPoolExecutor-5_1: done with 3
ThreadPoolExecutor-5_0: done with 1
main: results: [50.0, 40.0, 30.0, 20.0, 10.0]


It is also possible to schedule individual tasks:

In [9]:
from concurrent import futures
import threading
import time


def task(n):
    print('{}: sleeping {}'.format(
        threading.current_thread().name,
        n)
    )
    time.sleep(n)
    print('{}: done with {}'.format(
        threading.current_thread().name,
        n)
    )
    return n * 10.

ex = futures.ThreadPoolExecutor(max_workers=2)
print('main: starting')
f = ex.submit(task, 5)
print('main: future: {}'.format(f))
print('main: waiting for results')
result = f.result()
print('main: result: {}'.format(result))
print('main: future after result: {}'.format(f))

main: starting
ThreadPoolExecutor-7_0: sleeping 5
main: future: <Future at 0x146d500f4b50 state=running>
main: waiting for results
ThreadPoolExecutor-7_0: done with 5
main: result: 50.0
main: future after result: <Future at 0x146d500f4b50 state=finished returned float>


Invoking the ```result()``` method of a Future blocks until the task completes (either by returning a value or raising an exception), or is canceled. 

The results of multiple tasks can be accessed in the order the tasks were scheduled using ```map()```. If it does not matter what order the results should be processed, use ```as_completed()``` to process them as each task finishes.

In [11]:
from concurrent import futures
import random
import time


def task(n):
    t = random.randint(1,5)
    time.sleep(t)
    return (n, n * 10.)


ex = futures.ThreadPoolExecutor(max_workers=5)
print('main: starting')

wait_for = [
    ex.submit(task, i)
    for i in range(5, 0, -1)
]

for f in futures.as_completed(wait_for):
    print('main: result: {}'.format(f.result()))

main: starting
main: result: (1, 10.0)
main: result: (3, 30.0)
main: result: (4, 40.0)
main: result: (5, 50.0)
main: result: (2, 20.0)


To take some action when a task completed, without explicitly waiting for the result, use ```add_done_callback()``` to specify a new function to call when the Future is done. 

The callback should be a callable taking a single argument, the Future instance.

In [17]:
from concurrent import futures
import time


def task(n):
    print('task sleeping for {} seconds'.format(n))
    time.sleep(n)
    print('task (agument {}) done'.format(n))
    return n * 10


def done(fn):
    if fn.cancelled():
        print('{}: canceled'.format(fn.arg))
    elif fn.done():
        error = fn.exception()
        if error:
            print('{}: error returned: {}'.format(
                fn.arg, error))
        else:
            result = fn.result()
            print('{}: value returned: {}'.format(
                fn.arg, result))


if __name__ == '__main__':
    #Initiate the threads
    ex = futures.ThreadPoolExecutor(max_workers=2)
    
    #submit the threads to the function "task"
    print('main: starting')
    f = ex.submit(task, 5)
    f.arg = 5
    
    time.sleep(0.1)
    print("moving on I think I want to call task again")
    
    print("Calling task again: ",task(1))
    
    f.add_done_callback(done)
    result = f.result()

main: starting
task sleeping for 5 seconds
moving on I think I want to call task again
task sleeping for 1 seconds
task (agument 1) done
Calling task again:  10
task (agument 5) done
5: value returned: 50


### Process Pools

The ProcessPoolExecutor works in the same way as ThreadPoolExecutor, but uses processes instead of threads. This allows CPU-intensive operations to use a separate CPU and not be blocked by the CPython interpreter’s global interpreter lock.


In [23]:
from concurrent import futures
import os
import time

def task(n):
    time.sleep(n)
    return (n, os.getpid())

ex = futures.ProcessPoolExecutor(max_workers=2)
results = ex.map(task, range(5, 0, -1))
for n, pid in results:
    print('ran task {} in process {}'.format(n, pid))

ran task 5 in process 20521
ran task 4 in process 20524
ran task 3 in process 20524
ran task 2 in process 20521
ran task 1 in process 20521


As with the thread pool, individual worker processes are reused for multiple tasks. 

What happens if you comment out the sleep command?

### References
* https://pymotw.com/3/concurrency.html
* https://docs.python.org/3/library/concurrency.html
